[View in Colaboratory](https://colab.research.google.com/github/agnibrainhack/Sentiment_Detect/blob/master/CDAC_v_2.ipynb)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Sentiment.csv to Sentiment.csv


In [2]:
% reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import keras
from sklearn.utils import resample

In [0]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [65]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

df_majority = data[data.sentiment=='Negative']
df_minority = data[data.sentiment=='Positive']
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=8000,    # to match majority class
                                 random_state=123) # reproducible results
data = pd.concat([df_majority, df_minority_upsampled])
data.sentiment.value_counts()
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

14000
16986


In [66]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(BatchNormalization())
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, kernel_initializer=keras.initializers.he_uniform(seed=None)))
model.add(Dense(10,activation='relu', kernel_initializer=keras.initializers.he_uniform(seed=None)))
# model.add(Dropout(0.2))
model.add(Dense(2,activation='sigmoid', kernel_initializer=keras.initializers.he_uniform(seed=None)))
# model.add(BatchNormalization())
model.compile(loss = 'binary_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 28, 128)           256000    
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 128)           512       
_________________________________________________________________
lstm_12 (LSTM)               (None, 196)               254800    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                1970      
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 22        
Total params: 513,304
Trainable params: 513,048
Non-trainable params: 256
_________________________________________________________________
None


In [67]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.4, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9895, 28) (9895, 2)
(6598, 28) (6598, 2)


In [68]:
batch_size = 8
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 114s - loss: 0.4808 - acc: 0.7585
Epoch 2/7
 - 111s - loss: 0.3105 - acc: 0.8764
Epoch 3/7
 - 111s - loss: 0.2500 - acc: 0.9055
Epoch 4/7
 - 110s - loss: 0.2134 - acc: 0.9210
Epoch 5/7
 - 111s - loss: 0.1872 - acc: 0.9330
Epoch 6/7
 - 111s - loss: 0.1690 - acc: 0.9410
Epoch 7/7
 - 110s - loss: 0.1542 - acc: 0.9486


In [69]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.35
acc: 0.90


In [70]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 93.47517730496455 %
neg_acc 88.17610062893083 %


In [77]:
twt = ['Meetings: Because we are lovely and are sick and sick of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  230  39  40   5  40 888   5 888   6 209]]
negative


In [79]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'Emotion_Det.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)

print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/Emotion_Det.h5 


In [0]:
from google.colab import auth
auth.authenticate_user()


In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [84]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Emotion_Det.h5',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/content/saved_models/Emotion_Det.h5', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))


File ID: 1wnmSfY7ibeIuprGoGK18yV8wEd1yrEmP
